In [ ]:
#Run function created to run and prin shell command
# Run function taken from colab.research.google documentation for installation
def run (cmd):
  print('>> {}'.format(cmd))
  !{cmd}
  print('')

  # Install apache-beam
run('pip install apache-beam[interactive]')

#Installing Pandas, Matplotlib, MYSQL Connector
!pip install Pandas
!pip install matplotlib
!sudo apt update
!pip install mysql-connector-python
!pip install MySQL

import apache_beam as beam
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
import os
import sys
import csv

>> pip install apache-beam[interactive]
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.1/252.1 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# dataframe(df)-> 2 dimensional data structure like
# a 2 dimensional array or a table with rows and columns

try:
  df=pd.read_csv('https://media.githubusercontent.com/media/christiane-villaroel/credit-card-fraud-detection/main/dataset.csv')
  print("File read successfully!\nYay!")

except pd.errors.ParserError as e:
  print(f'error reading file: {e}')
  with open('https://media.githubusercontent.com/media/christiane-villaroel/credit-card-fraud-detection/main/dataset.csv','r') as f:
    for line in f:
      print(line, end ='')

# saving dataframe(df) to a csv file on colab
df.to_csv('dataset.csv', index=False,)
print("File saved successfully!")

File read successfully!
Yay!
File saved successfully!


In [ ]:
data= df.drop(['nameOrig','type','isFlaggedFraud'],axis=1)
data['amountDest']= (data['newbalanceDest']-data['oldbalanceDest']).abs()
data['amountOrig']= data['newbalanceOrig']-data['oldbalanceOrg'].abs()
data.drop(['newbalanceDest','oldbalanceDest','newbalanceOrig','oldbalanceOrg'],axis=1,inplace=True)

data.head()

,step,amount,nameDest,isFraud,amountDest,amountOrig
0,1,9839.64,M1979787155,0,0.0,-9839.64
1,1,1864.28,M2044282225,0,0.0,-1864.28
2,1,181.00,C553264065,1,0.0,-181.00
3,1,181.00,C38997010,1,21182.0,-181.00
4,1,11668.14,M1230701703,0,0.0,-11668.14


In [ ]:
# Sample column definitions based on the CSV file
column_definitions = {
    'step': 'INT',
    'type': 'VARCHAR(255)',
    'amount': 'FLOAT',
    'nameOrig': 'VARCHAR(255)',
    'oldbalanceOrg': 'FLOAT',
    'newbalanceOrig': 'FLOAT',
    'nameDest': 'VARCHAR(255)',
    'oldbalanceDest': 'FLOAT',
    'newbalanceDest': 'FLOAT',
    'isFraud': 'INT',
    'isFlaggedFraud': 'INT'
}

In [ ]:
# Create the table creation SQL query
def create_table_query(table_name, columns):
    cols = ",\n".join([f"{col_name} {data_type}" for col_name, data_type in columns.items()])
    query = f"CREATE TABLE IF NOT EXISTS {table_name} (\n{cols}\n);"
    return query

In [ ]:
config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Andrewcool318',
    'database': 'dailytest'
}

In [ ]:
# Connect to MySQL database
def connect_to_mysql(element):
  try:
      connection = mysql.connector.connect(**config)
      cursor = connection.cursor()

      # Create the table
      table_name = 'log_data'
      query = create_table_query(table_name, column_definitions)
      cursor.execute(query)

      # Insert DataFrame into MySQL table using batch inserts
      batch_size = 100  # You can adjust this batch size based on your memory capacity and performance
      for start in range(0, len(element), batch_size):
          end = start + batch_size
          batch = df.iloc[start:end]

          insert_query = f"""
          INSERT INTO {table_name} ({', '.join(column_definitions.keys())})
          VALUES ({', '.join(['%s'] * len(column_definitions))});
          """

          cursor.executemany(insert_query, [tuple(row) for row in batch.itertuples(index=False, name=None)])

      connection.commit()

  except Error as e:
      print(f"Error: {e}")
  finally:
      if 'connection' in locals() and connection.is_connected():
          cursor.close()
          connection.close()
          print("CSV data has been imported into MySQL database successfully.")

Error: 2003 (HY000): Can't connect to MySQL server on 'localhost:3306' (99)


In [ ]:
csv_strings = df.to_csv(index=False, header=True).split('\n')
 # creating pipeline
pipeline = beam.Pipeline()

outputs = (
    pipeline
    | 'Read CSV' >> beam.Create(csv_strings)
    | 'Split CSV' >> beam.Map(lambda s: s.split(','))
    | 'Dictionary' >> beam.Map(lambda row: dict(zip(column_definitions.keys(), row)))
    | 'Batch Elements' >> beam.BatchElements(min_batch_size=10,max_batch_size=1000)
    | 'Write to MYSQL' >> beam.ParDo(connect_to_mysql)

)

In [ ]:
# Run pipeline and runs outputs commands: Read, Write and Print output CSV
pipeline.run()